In [5]:
from cobra.io import read_sbml_model

In [7]:
model=read_sbml_model('models/yeast_lyco.xml')

# Maximum theoretical yield of lycopene using default medium

**Calculating the maximum productivity and maximum theoretical yield in mmol/mmol:**

In [9]:
with model:
    model.objective = model.reactions.CrtI #Setting the objective to the reaction, which is producing Lycopene
    lycopene_production = model.optimize().objective_value
    print('Maximum productivity of lycopene =', lycopene_production, 'mmol/gDW*h')

    glc_uptake_flux = model.reactions.r_1714.flux 
    max_yield = lycopene_production / (-1*glc_uptake_flux)
    print('Maximum theoretical yield =', max_yield, 'mmol-lyco/mmol-glc')

Maximum productivity of lycopene = 0.0871195699738864 mmol/gDW*h
Maximum theoretical yield = 0.0871195699738864 mmol-lyco/mmol-glc


**Calculating the maximum productivity and maximum theoretical yield in C-mol/C-mol:**

In [3]:
# Factors for conversion from mol to C-mol
cv_glc = 6   # Glucose
cv_lyco = 40 # Lycopene

lyco_prod_cmol = (lycopene_production)/(cv_lyco)
lyco_yield_cmol = (max_yield*cv_lyco)/(cv_glc)
print('Maximum productivity of lycopene =', lyco_prod_cmol, 'cmol-lyco/gDW*h')
print('Maximum theoretical yield =', lyco_yield_cmol, 'cmol-lyco/cmol-glc')

Maximum productivity of lycopene = 0.00217798924934716 cmol-lyco/gDW*h
Maximum theoretical yield = 0.5807971331592426 cmol-lyco/cmol-glc


The theoretical production and yield of lycopene on glucose is fairly low.\
Summarized in mmol units:

- Maximum productivity = $0.087 \frac{mmol-lyco}{g DW \cdot h}$

- Maximum yield = $0.087 \frac{mmol-lyco}{mmol-glc}$

And in C-mol units:

- Maximum productivity = $0.002 \frac{cmol-lyco}{g DW \cdot h}$

- Maximum yield = $0.581 \frac{cmol-lyco}{cmol-glc}$

#### Increasing the glucose concentration in order to increase lycopene production:

Firstly, we check the medium composition:

In [4]:
# Checking medium composition via the extracellular exchange reactions
for m in model.medium.items():
    print(model.reactions.get_by_id(m[0]), "|", model.reactions.get_by_id(m[0]).name, "| conc. =", m[1])

r_1654: s_0420[e] <=>  | ammonium exchange | conc. = 1000.0
r_1714: s_0565[e] <=>  | D-glucose exchange | conc. = 1.0
r_1832: s_0796[e] <=>  | H+ exchange | conc. = 1000.0
r_1861: s_0925[e] <=>  | iron(2+) exchange | conc. = 1000.0
r_1992: s_1277[e] <=>  | oxygen exchange | conc. = 1000.0
r_2005: s_1324[e] <=>  | phosphate exchange | conc. = 1000.0
r_2020: s_1374[e] <=>  | potassium exchange | conc. = 1000.0
r_2049: s_1438[e] <=>  | sodium exchange | conc. = 1000.0
r_2060: s_1468[e] <=>  | sulphate exchange | conc. = 1000.0
r_2100: s_0805[e] <=>  | water exchange | conc. = 1000.0
r_4593: s_4200[e] <=>  | chloride exchange | conc. = 1000.0
r_4594: s_4201[e] <=>  | Cu2(+) exchange | conc. = 1000.0
r_4595: s_4202[e] <=>  | Mn(2+) exchange | conc. = 1000.0
r_4596: s_4203[e] <=>  | Zn(2+) exchange | conc. = 1000.0
r_4597: s_4204[e] <=>  | Mg(2+) exchange | conc. = 1000.0
r_4600: s_4199[e] <=>  | Ca(2+) exchange | conc. = 1000.0


#### Increasing glucose concentration and investigating the effects on growth rate:

In [11]:
medium_glc_ex = model.reactions.get_by_id('r_1714')

In [12]:
medium = model.medium
with model: #modification are donw locally within the indent
    growth_rate = model.optimize().objective_value
    print("At glucose concentration = {}:".format(medium[medium_glc_ex.id]), "Maximum theoretical growth rate =", growth_rate, "h^-1")
    
    medium[medium_glc_ex.id] = 1000 #glucose level is changed 
    model.medium = medium
    
    growth_rate = model.optimize().objective_value
    print("At glucose concentration = {}:".format(medium[medium_glc_ex.id]), "Maximum theoretical growth rate =", growth_rate, "h^-1")    

At glucose concentration = 1.0: Maximum theoretical growth rate = 0.08374775763992785 h^-1
At glucose concentration = 1000: Maximum theoretical growth rate = 19.81683025946431 h^-1


#### Increasing the effects of glucose on lycopene production:

In [23]:
medium = model.medium
with model:
    model.objective = model.reactions.CrtI #Setting the objective to the reaction, which is producing Lycopene
    lycopene_production = model.optimize().objective_value
    print("At glucose concentration = {}:".format(medium[medium_glc_ex.id]))
    print('Maximum theoretical productivity of lycopene =', lycopene_production, 'mmol/(gDW*h^-1)')
    
    glc_uptake_flux = model.reactions.r_1714.flux 
    max_yield = lycopene_production / (-1*glc_uptake_flux)
    print('Maximum theoretical yield =', max_yield, 'mmol-lyco/mmol-glc\n')
    
    medium[medium_glc_ex.id] = 1000
    model.medium = medium

    lycopene_production = model.optimize().objective_value
    glc_uptake_flux = model.reactions.r_1714.flux
    max_yield = lycopene_production / (-1*glc_uptake_flux)
    print("At glucose concentration = {}:".format(medium[medium_glc_ex.id]))
    print('Maximum theoretical productivity of lycopene =', lycopene_production, 'mmol/(gDW*h*^-1)')
    print('Maximum theoretical yield =', max_yield, 'mmol-lyco/mmol-glc\n')

At glucose concentration = 1.0:
Maximum theoretical productivity of lycopene = 0.08711956997388663 mmol/(gDW*h^-1)
Maximum theoretical yield = 0.08711956997388663 mmol-lyco/mmol-glc

At glucose concentration = 1000:
Maximum theoretical productivity of lycopene = 25.825440917107475 mmol/(gDW*h*^-1)
Maximum theoretical yield = 0.04014908299958047 mmol-lyco/mmol-glc



Increasing the external glucose concentration 1000-fold can theoreticlaly increase the lycopene productivity from $0.087 \frac{mmol}{gDW \cdot h}$ to approximately $25.825 \frac{mmol}{gDW \cdot h}$.\
This is a 296-fold increase!

However, this observation is not the complete picture, as increasing glucose concentration decreases the potential yield of lycopene from $0.087 \frac{mmol-lyco}{mmol-glc}$ to $0.040 \frac{mmol-lyco}{mmol-glc}$.